# Multithreading und Multiprocessing
In diesem Abschnitt wird der Unterschied zwischen Threads und Prozessen erklärt. Weiterhin werden Umsetzungen in Python und Besonderheiten der Programmiersprache hinsichtlich der Parallelisierung gezeigt. Zuletzt wird ein alternatives Konzept der Nebenläufigkeit präsentiert.

In [ ]:
import time
import threading
import multiprocessing
import asyncio
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

from tui_dsmt.parallel import mpi_run

## Inhaltsverzeichnis
- [Threads und Prozesse](#Threads-und-Prozesse)
- [Threads in Python und GIL](#Threads-in-Python-und-GIL)
- [Prozesse in Python](#Prozesse-in-Python)
- [Coroutinen](#Coroutinen)
- [Message Parsing Interface (MPI)](#Message-Parsing-Interface-MPI)
- [Zusammenfassung](#Zusammenfassung)

## Threads und Prozesse
Threads und Prozesse sind grundlegende Konzepte, die zur nebenläufigen Ausführung von mehreren Aufgaben auf einem Computer verwendet werden.

Ein Prozess ist eine eigenständige Ausführungseinheit, die einen eigenen Speicherbereich zugewiesen bekommt. Prozesse werden in der Regel vom Betriebssystem verwaltet, das sich um eben diese Zuweisung und die Aufteilung der Rechenzeit auf die existieren Prozesse kümmert. Die getrennten Speicherbereiche sorgen dafür, dass Prozesse voneinander isoliert ablaufen und sich gegenseitig - bei ausreichend vorhandener Rechenzeit - nicht beeinflussen.

Ein Thread hingegen ist eine Ausführungseinheit innerhalb eines Prozesses. Die Threads eines Prozesses teilen sich gemeinsam den Speicherbereich des Prozesses und können nach Belieben aus diesem lesen und schreiben. Threads gelten außerdem als *leichter*, weil sie weniger Overhead erzeugen, da kein separater Speicher reserviert werden muss, und weil sie schneller zu erzeugen und zu zerstören sind als vollwertige Prozesse. Die Zuteilung der Rechenleistung erfolgt bei Threads in der Regel auch durch das Betriebssystem, es existieren allerdings auch [andere Implementierungen](https://en.wikipedia.org/wiki/Green_thread) mit eigenen Vor- und Nachteilen.

Die Zusammenarbeit mehrerer Threads ist in der Regel einfach, da ein Datensatz oder Zwischenergebnisse nur einmal im Speicher liegen muss und alle Threads nach Belieben auf diesen Speicher zugreifen können. Das Teilen von Speicher macht die Algorithmen jedoch anfälliger gegenüber Korruption und Race Conditions. Die Zusammenarbeit mehrerer Prozesse muss dagegen mit Hilfe einer Interprozesskommunikationmethode realisiert werden, um Teile des Speicherinhalts wie Daten oder Zwischenergebnisse an andere Prozesse zu versenden. Interprozesskommunikation ist dabei immer langsamer als geteilter Speicher.

Die Verwendung mehrerer Prozesse ist beim Aufteilen der parallelen Arbeit auf mehrere, durch ein Netzwerk verbundene Computer alternativlos, da kein gemeinsamer Hauptspeicher existiert.

## Threads in Python und GIL
In Python können Threads mit dem `threading`-Modul erstellt werden. Das nachfolgende Beispiel erstellt zwei Threads, die parallel arbeiten und jeweils eine bestimmte Sequenz ausgeben.

In [ ]:
def print_seq(sequence):
    for i in sequence:
        print(i)
        time.sleep(0.5)

# Erzeugen der Threads
thread1 = threading.Thread(target=print_seq, args=(range(1, 5),))
thread2 = threading.Thread(target=print_seq, args=('abcd',))

# Starten der Threads (im Hintergrund)
thread1.start()
thread2.start()

# Warten auf Beenden der Threads / Funktionen
thread1.join()
thread2.join()

Erwartungsgemäß sollten abwechselnd Ziffern und Buchstaben ausgegeben werden. In der Realität ist die Zuweisung der Rechenzeit aber von so vielen Faktoren abhängig, dass die genaue Reihenfolge nicht vorhersehbar ist. Beim wiederholten Ausführen werden Sie feststellen, dass kleine Abweichungen wie das Vertauschen von Buchstabe und Ziffer und in seltenen Fällen auch zwei Einträge pro Zeile gefolgt von einer Leerzeile vorkommen.

In Abhängigkeit der vorhandenen Hardware verlangsamt es die Berechnung möglicherweise, zu viele Threads anzulegen. Python hält mit dem `ThreadPoolExecutor` auch dafür eine Lösung bereit: Diesem kann eine beliebige Anzahl an Aufgaben übergeben werden, die in einer festen Anzahl an Threads abgearbeitet werden. Sobald ein Thread mit einer Aufgabe fertig ist, übernimmt er die nächste. Durch die Verwendung als Kontext-Manager wird die Shutdown Methode nach Ende des Blocks automatisch aufgerufen und die Übergabe weiterer Aufgaben blockiert.

In [ ]:
with ThreadPoolExecutor(max_workers=1) as executor:
    executor.submit(print_seq, range(1, 5))
    executor.submit(print_seq, 'abcd')

In Python existiert zudem mit dem sogenannten *Global Interpreter Lock* (GIL) ein Mechanismus, der dafür sorgt, dass immer nur ein Thread Python-Code ausführen kann. Dies bedeutet, dass selbst in einem Multithreading-Programm mit mehreren Threads, nur ein Thread zur gleichen Zeit auf dem Prozessor ausgeführt wird. Die nachfolgenden Zellen verdeutlichen das Problem.

In [ ]:
def cpu_bound_task():
    counter = 0
    while counter < 10_000_000:
        counter += 1

In [ ]:
%%timeit
# Sequentielle Ausführung von zwei Durchläufen
cpu_bound_task()
cpu_bound_task()

In [ ]:
%%timeit
# Parallele Ausführung in zwei Threads
threads = [threading.Thread(target=cpu_bound_task) for _ in range(2)]
for t in threads: t.start()
for t in threads: t.join()

Erwartungsgemäß sollte die parallele Ausführung nur etwa die Hälfte der Zeit benötigen, da die Arbeit auf zwei Threads verteilt wird. Was in vielen Programmiersprachen genau so funktioniert, benötigt durch den GIL in Python dennoch die selbe Zeit wie die sequentielle Ausführung. Die Verwendung von Threads lohnt sich mit Python daher nur, wenn die Threads nicht durch den Prozessor limitiert werden, sondern z.B. durch das Lesen von Daten von der Festplatte oder durch Kommunikation mit dem Netzwerk.

Die [Entfernung des GIL](https://heise.de/-9655663 ) ist aktuell ein Ziel bei der Weiterentwicklung der Sprache.

## Prozesse in Python

Um parallele Ausführungen mit mehreren Prozessen in Python zu demonstrieren, verwenden wir das `multiprocessing`-Modul. Dieses Modul ermöglicht es uns, unabhängige Prozesse zu erstellen, die in separaten Speicherbereichen jeweils einen eigenen Python-Interpreter ausführen. Die nachfolgende Zelle demonstriert die Verwendung von Prozessen anstelle von Threads:

In [ ]:
def print_seq(sequence):
    for i in sequence:
        print(i)
        time.sleep(0.5)

# Erzeugen der Prozesse
proc1 = multiprocessing.Process(target=print_seq, args=(range(1, 5),))
proc2 = multiprocessing.Process(target=print_seq, args=('abcd',))

# Starten der Prozesse (im Hintergrund)
proc1.start()
proc2.start()

# Warten auf Beenden der Prozesse
proc1.join()
proc2.join()

Da die Prozesse keinen gemeinsamen Speicher verwenden, können auch keine Variablen zwischen den Aufgaben geteilt werden, um Ergebnisse zu kommunizieren. Aus dem `multiprocessing`-Modul stehen aber spezielle Objekte bereit, um diesem Umstand zu begegnen:

In [ ]:
def in_process(result):
    result[1] = 'a'

result_dict = multiprocessing.Manager().dict()

proc = multiprocessing.Process(target=in_process, args=(result_dict,))
proc.start()
proc.join()

result_dict.items()

An den nachfolgenden Zellen können Sie nachvollziehen, dass die Prozesse tatsächlich unabhängig voneinander arbeiten und insbesondere nicht durch den Global Interpreter Lock (GIL) eingeschränkt sind. Die parallel ausgeführten Prozesse benötigen für die gleichen Operationen in etwas über die Hälfte der Zeit.

In [ ]:
%%timeit
# Sequentielle Ausführung von zwei Durchläufen
cpu_bound_task()
cpu_bound_task()

In [ ]:
%%timeit
# Parallele Ausführung in zwei Prozessen
procs = [multiprocessing.Process(target=cpu_bound_task) for _ in range(2)]
for p in procs: p.start()
for p in procs: p.join()

Auch für Prozesse steht ein Executor bereit:

In [ ]:
with ProcessPoolExecutor(max_workers=1) as executor:
    executor.submit(print_seq, range(1, 5))
    executor.submit(print_seq, 'abcd')

## Coroutinen
Python unterstützt außerdem Coroutinen, die einen speziellen Ansatz der Nebenläufigkeit umsetzen. In Threads ablaufende Aufgaben bekommen Rechenzeit vom Betriebssystem zugewiesen und können demnach jederzeit unterbrochen werden. Coroutinen hingegen verzichten an geeigneten Stellen freiwillig auf Rechenzeit und geben die Kontrolle an andere Coroutinen ab, während sie alle in einem gemeinsamen Thread laufen.

Coroutinen besitzen damit noch weniger Overhead als Threads. Sie benötigen nur Speicher für ihren Funktionsaufruf und den aktuellen Stack und auch Kontextwechsel sind deutlich schneller, sodass man mit ausreichend Arbeitsspeicher problemlos zehntausende von ihnen innerhalb einer Anwendung erzeugen kann. Da alle Coroutinen in einem gemeinsamen Thread laufen, teilen sie sich aber Eigenschaften mit dem GIL: Tatsächlich Python-Code ausführen kann immer nur eine einzige Coroutine, sodass der Einsatz nur bei Aufgaben dienlich ist, die einen großen Teil ihrer Zeit mit dem *Warten* auf das Netzwerk oder Festspeicher verbringen.

Geeignete Stellen für das Unterbrechen ist also genau dieses Warten auf Objekte, die langsamer als der Prozessor sind. Ein weiterer Vorteil des Aussuchens der Unterbrechung liegt weiterhin in weniger Seiteneffekten, die auftreten können, und einer verbesserten Lesbarkeit des Codes gegenüber Threads: Der Quellcode lässt sich überwiegend wie sequentieller Code lesen, auch wenn er tatsächlich Nebenläufigkeit erlaubt.

In Python steht in der Standardinstalltion für Coroutinen das Modul `asyncio` bereit, das zusammen mit den Schlüsselwörtern `async` und `await` die Programmierung von Coroutinen erlaubt. Der Unterschied einer normalen zu einer asynchronen Methode ist dabei, dass der Aufruf zunächst normal aussieht, allerdings ein Objekt vom Typ `coroutine` zurückgibt. Innerhalb dieser Methoden (und innerhalb von Jupyter Zellen) kann außerdem das Schlüsselwort `await` verwendet werden, um auf das Ergebnis einer Coroutine zu warten. Es markiert gleichzeitig die Möglichkeit zur Übernahme von Rechenzeit durch andere Coroutinen.

In [ ]:
async def async_print_seq(sequence):
    for i in sequence:
        print(i)
        await asyncio.sleep(0.5)

async_print_seq(range(1, 5))

Der Aufruf mehrere Funktionen, die parallel ablaufen, kann dann beispielsweise mit `gather` erfolgen:

In [ ]:
await asyncio.gather(*(
    async_print_seq(range(1, 5)),
    async_print_seq('abcd')
))

**Verständnisfrage**: Bei der Verwendung von Threads und Prozessen wurden gelegentlich
- Ziffer und Buchstabe in ihrer Reihenfolge vertauscht oder
- zwei Werte pro Zeile ausgegeben.

Kann das mit Coroutinen ebenfalls vorkommen?

## Message Parsing Interface (MPI)
MPI ist ein Standard für die parallele Programmierung auf verteilten Systemen mit lokalem Speicher. Entwickelt wurde der Standard in den 90er Jahren mit dem Ziel, eine portable und skalierbare Möglichkeit für parallele Programmierung zu schaffen, sodass es sich sowohl in einem durch ein Netzwerk verbundenes Cluster-System wie auch lokal mit mehreren Prozessen verwenden lässt. MPI stellt insbesondere Möglichkeiten bereit, um verschiedene Arten Nachrichten zwischen Prozessen auszutauschen, und gilt daher als sehr flexibel. So sind nicht nur Punkt-zu-Punkt Nachrichten möglich, sondern auch Broadcasts, Barrieren und Remote Memory Access.

Anwendung findet MPI daher in vielen verschiedenen Bereichen, insbesondere in der Simulation physikalischer Modelle, wie Wetter- und Klimamodellierung, Molekulardynamik oder Strömungsmechanik. Für Python existieren mehrere Bibliotheken, welche die Funktionalität von MPI verfügbar machen. Im Folgenden sollen kurz die relevantesten Operationen mit `mpi4py` gezeigt werden.

Zum Starten mehrerer Prozesse, die mit Hilfe von MPI zusammenarbeiten, muss MPI auf allen beteiligten Systemen installiert sein. Weitere Systeme lassen sich dann zum Beispiel über SSH zum Cluster hinzufügen, das Ausführen ist aber auch auf dem lokalen System mit mehreren Prozessen möglich. Zum Starten wird in der Regel `mpiexec` oder `mpirun` verwendet. Um in einem Terminal vier Python Prozesse zu starten, die untereinander mit MPI kommunizieren, kann das nachfolgende Kommando verwendet werden:

```bash
mpiexec -n 4 python3 script.py
```

Da wir im Folgenden die Jupyter Umgebung nicht verlassen wollen, steht eine Funktion `mpi_run` bereit, die einen lokalen Cluster erzeugt und **eine** übergebene Python Funktion ausführt. Die nachfolgende Zelle zeigt zudem die Verwendung von `Get_rank()`, das die ID des aktuellen Prozess zurückgibt, und `Get_size()`, das die Anzahl aller Prozesse liefert. Da alle Prozesse den selben Code ausführen, lassen sich mit diesen Funktionen Fallunterscheidungen treffen, um verschiedenen Prozessen verschiedene Aufgabenteile zuzuweisen.

In [ ]:
def mpi_fun():
    from mpi4py import MPI
    comm = MPI.COMM_WORLD
    rank, size = comm.Get_rank(), comm.Get_size()

    return f'Hallo Welt von Prozess {rank} von {size}'

mpi_run(mpi_fun)

Nachrichten zwischen zwei spezifischen Prozessen lassen sich mit den Funktionen `send` und `recv` austauschen.

In [ ]:
def mpi_fun():
    from mpi4py import MPI
    import random

    comm = MPI.COMM_WORLD
    rank, size = comm.Get_rank(), comm.Get_size()

    if rank == 0:
        random_num = random.randint(1, 20)
        comm.send(random_num, dest=1)
        return f'Prozess {rank} schickt {random_num}'

    if rank == 1:
        random_num = comm.recv(source=0)
        return f'Prozess {rank} empfängt {random_num}'

mpi_run(mpi_fun, num_proc=2)

Es lassen sich aber mit `bcast` auch Nachrichten an alle (anderen) Teilnehmer schicken. Die nachfolgende Zelle demonstriert zudem eine weitere Funktion von `mpi4py`: Es lassen sich nicht nur primitive Datentypen verschicken. Komplexere Typen werden automatisch mit Pickle serialisiert und deserialisiert, sodass auch das Versenden von Objekten, Listen, etc. möglich ist.

In [ ]:
def mpi_fun():
    from mpi4py import MPI
    import random

    comm = MPI.COMM_WORLD
    rank, size = comm.Get_rank(), comm.Get_size()

    # Objekt initialisieren
    random_obj = None

    # Objekt auf erstem Prozess vorbereiten
    if rank == 0:
        random_obj = {
            'zahl': random.randint(1, 20),
            'buchstabe': random.choice('abcdefgh')
        }

    # Broadcast aufrufen, Quelle ist Prozess 0
    random_obj = comm.bcast(random_obj, root=0)
    return f'Prozess {rank} empfängt {random_obj}'

mpi_run(mpi_fun, num_proc=3)

Barrieren sind Synchronisationspunkte, an denen jeder Prozess wartet, bis die Barriere von allen Prozessen erreicht wurde.

In [ ]:
def mpi_fun():
    from mpi4py import MPI
    import time

    comm = MPI.COMM_WORLD
    rank, size = comm.Get_rank(), comm.Get_size()

    # Startzeit speichern
    start = time.time()

    # unterschiedlich lang schlafen
    sleep_time = rank * 1.5
    time.sleep(sleep_time)

    # Barriere einsetzen
    comm.Barrier()

    # verbrachte Zeit zurückgeben
    time_spent = time.time() - start
    return f'Prozess {rank} schlief {sleep_time}s und endete nach {time_spent}s'

mpi_run(mpi_fun, num_proc=3)

MPI stellt viele weitere Funktionen bereit, unter anderem Locks, Windows / Remote Memory Access und verteilte I/O Operationen, die allesamt verwendet werden können, um MPI Programme effizienter zu gestalten, und ebenfalls von `mpi4py` unterstützt werden.

## Zusammenfassung
In Python stehen verschiedene Methoden bereit, um Nebenläufigkeit zu erzeugen. Das Erzeugen von Prozessen und Threads reicht aber lang nicht aus, um produktiv parallele Programme zu entwickeln. So muss sich um eine korrekte Aufgabenteilung gekümmert, Prozesse synchronisiert, Zwischenergebnisse kommuniziert und nach Möglichkeit eine Lastverteilung vorgenommen werden. MPI ist eine Bibliothek, welche die dafür notwendigen Funktionen für eine Vielzahl von Programmiersprachen standardisiert.

Beim Ablauf paralleler Programme können außerdem Probleme auftreten, die bei sequentiellen Programmen nicht vorkommen können, so zum Beispiel Race Conditions, Deadlocks oder auch Konsistenzprobleme (bspw. Lost Updates). Nachfolgend sollen daher Modelle präsentiert werden, die bei Zutreffen bestimmer Annahmen die Programmierung vereinfachen und gleichzeitig derartige Fehler vermeiden.